In [ ]:
import mitsuba as mi
mi.set_variant("llvm_ad_rgb")
import drjit as dr

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmap_diff
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import os
base_dir = 'estimator_comparison_envmap_conductor'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    
mi.Thread.thread().logger().set_log_level(mi.LogLevel.Warn)

def produce_plots(methods, method_names, directory, gradients=False):
    # Suitable values to compare gradients computed with all different methods (determined manually)
    scales = np.array([20, 10, 5, 2])

    for method, method_name in zip(methods, method_names):
        fig, axes = plt.subplots(ncols=len(alphas), nrows=1, figsize=(12,4))
        for idx_alpha, _ in enumerate(alphas):
            name = "a_{:02d}".format(idx_alpha)
            path = '{}/{}/{}/{}.exr'.format(base_dir, directory, method, name)
            ax = axes[idx_alpha]
            ax.set_xticks([]); ax.set_yticks([])
            data = np.array(mi.Bitmap(path)).astype('float32')
            if gradients:
                vminmax = scales[idx_alpha]
                data_ = data[:,:,0] if len(data.shape) == 3 else data
                im = ax.imshow(data_, cmap='diff', vmin=-vminmax, vmax=+vminmax)
            else:
                data = np.clip(data**(1/2.2), 0.0, 1.0) # Crude gamma correction
                im = ax.imshow(data, cmap='gray')

        fig.suptitle(method_name, y=0.98, size=13, weight='bold')
        fig.text(0.125, 0.85, 'Surface roughness --->',
                 ha='left', size=14, weight='bold')
        for idx_alpha, alpha in enumerate(alphas):
            axes[idx_alpha].set_title(r"$\alpha={}$".format(alpha), size=14)
        outname = '{}/{}/{}.jpg'.format(base_dir, directory, method)
        plt.savefig(outname, dpi=150, pad_inches=0.1, bbox_inches='tight')
        plt.show()

In [ ]:
# We look at a the simple test scene of a rough conductor plane, lit by an environment map.
# This allows us to judge the effectiveness of the different MC estimators at various
# combinations of emitter concentration and surface roughness parameters.
# No discontinuities are present in this scene.

# Scene parameters varied in each configuration
alpha_key = 'plane.bsdf.alpha.data'

# Values to consider in the comparison
alphas = [0.02, 0.05, 0.1, 0.2]

# Sampels per pixel for all renderings
spp = 128

primal_methods = [
    'primal_bs',
    'primal_es',
    'primal_mis',
]
primal_method_names = [
    'Primal BSDF sampling',
    'Primal emitter sampling',
    'Primal MIS (BSDF + emitter sampling)',
]

diff_methods = [
    'es_detached',
    
    'bs_detached',
    'bs_attached',
    
    'mis_detached_detached',
    'mis_attached_attached',
    'mis_detached_attached',
    'mis_attached_detached',
    
    'bs_detached_diff',
    'mis_detached_detached_diff',
]
diff_method_names = [
    'Detached emitter sampling',
    
    'Detached BSDF sampling',
    'Attached BSDF sampling',
    
    'Detached MIS weights, detached BSDF sampling, detached emitter sampling',
    'Attached MIS weights, attached BSDF sampling, detached emitter sampling',
    'Detached MIS weights, attached BSDF sampling, detached emitter sampling (BIASED!)',
    'Attached MIS weights, detached BSDF sampling, detached emitter sampling',
    
    'Detached diff. BSDF sampling',
    'Detached MIS weights, detached diff. BSDF sampling, detached emitter sampling',
]

In [ ]:
# #####################
# # Primal estimators #
# #####################

for method, method_name in zip(primal_methods, primal_method_names):
    print("* {}".format(method_name))
    method_dir = "{}/primal/{}".format(base_dir, method)
    if not os.path.exists(method_dir):
        os.makedirs(method_dir)
        
    integrator = mi.load_dict({'type': 'estimator_comparison', 'method': method, 'hide_emitters': True})
        
    scene = mi.load_file('scenes/conductor_plane_envmap.xml')
    params = mi.traverse(scene)
    params.keep([alpha_key])
    texture_shape = params[alpha_key].shape
    
    for idx_alpha, alpha in enumerate(alphas):
        print("  - {}/{}".format(idx_alpha + 1, len(alphas)), end='\r')
        params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*alpha)
        params.update()

        image = mi.render(scene, params, integrator=integrator, seed=0, spp=spp)
        outname = "{}/a_{:02d}.exr".format(method_dir, idx_alpha)
        mi.util.convert_to_bitmap(image, uint8_srgb=False).write(outname)
    print("")
    
produce_plots(primal_methods, primal_method_names, 'primal')

In [ ]:
######################################
# Finite differences (FORWARD mode)  #
######################################
# Produce a gradient image as a result.

eps = 1e-4

for method, method_name in zip(['fd'], ['Finite differences']):
    print("* {}".format(method_name))
    method_dir = "{}/gradients_forward/{}".format(base_dir, method)
    if not os.path.exists(method_dir):
        os.makedirs(method_dir)

    integrator = mi.load_dict({'type': 'estimator_comparison', 'method': 'primal_mis', 'hide_emitters': True})
    
    scene = mi.load_file('scenes/conductor_plane_envmap.xml')
    params = mi.traverse(scene)
    params.keep([alpha_key])
    texture_shape = params[alpha_key].shape
    
    for idx_alpha, alpha in enumerate(alphas):
        print("  - {}/{}".format(idx_alpha + 1, len(alphas)), end='\r') 
        params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*alpha)
        params.update()

        # Render first image
        image_0 = mi.render(scene, params, integrator=integrator, seed=0, spp=512)

        # Apply FD
        params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*(alpha + eps))
        params.update()

        # Render second image
        image_1 = mi.render(scene, params, integrator=integrator, seed=0, spp=512)

        # Save output gradient image
        image_grad = (image_1 - image_0) / eps
        outname = "{}/a_{:02d}.exr".format(method_dir, idx_alpha)
        mi.util.convert_to_bitmap(image_grad, uint8_srgb=False).write(outname)
            
produce_plots(['fd'], ['Finite differences'], 'gradients_forward', gradients=True)

In [ ]:
######################################
# Gradient estimators (FORWARD mode) #
######################################
# These propagate a scalar input gradient (in the surface roughness) to the output pixels,
# i.e. produce a gradient image as a result.

for method, method_name in zip(diff_methods, diff_method_names):
    print("* {}".format(method_name))
    method_dir = "{}/gradients_forward/{}".format(base_dir, method)
    if not os.path.exists(method_dir):
        os.makedirs(method_dir)
        
    integrator = mi.load_dict({'type': 'estimator_comparison', 'method': method, 'hide_emitters': True})
        
    scene = mi.load_file('scenes/conductor_plane_envmap.xml')
    params = mi.traverse(scene)
    params.keep([alpha_key])
    texture_shape = params[alpha_key].shape
    
    for idx_alpha, alpha in enumerate(alphas):
        print("  - {}/{}".format(idx_alpha + 1, len(alphas)), end='\r') 
        params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*alpha)

        # Diff. input parameter
        pi = mi.Float(0.0)
        dr.enable_grad(pi)
        dr.set_grad(pi, 1.0)
        params[alpha_key] += pi
        params.update()

        image_grad = mi.Float(0.0)
        if 'diff' in method:
            # Differential sampling strategy, use antithetic sampling.
            # Note that we use the same seed twice, and use half the number of samples for each pass.
            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp//2, antithetic_pass=False)
            dr.forward(pi)
            image_grad = dr.grad(image)

            params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*alpha)
            params[alpha_key] += pi
            params.update()

            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp//2, antithetic_pass=True)
            dr.forward(pi)
            image_grad += dr.grad(image)

            # Average both passes
            image_grad *= 0.5
        else:
            # Produce differentiable rendering
            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp)
            # And propagate derivatives forwards through it
            dr.forward(pi)
            image_grad = dr.grad(image)

        # Save output gradient image
        outname = "{}/a_{:02d}.exr".format(method_dir, idx_alpha)
        mi.util.convert_to_bitmap(image_grad, uint8_srgb=False).write(outname)
            
produce_plots(diff_methods, diff_method_names, 'gradients_forward', gradients=True)

In [ ]:
######################################
# Gradient estimators (REVERSE mode) #
######################################
# These propagate an output image gradient to the (textured) scene input parameters,
# i.e. the result are texture gradient images.

for method, method_name in zip(diff_methods, diff_method_names):
    print("* {}".format(method_name))
    method_dir = "{}/gradients_reverse/{}".format(base_dir, method)
    if not os.path.exists(method_dir):
        os.makedirs(method_dir)
        
    integrator = mi.load_dict({'type': 'estimator_comparison', 'method': method, 'hide_emitters': True})
        
    scene = mi.load_file('scenes/conductor_plane_envmap.xml')
    params = mi.traverse(scene)
    params.keep([alpha_key])
    texture_shape = params[alpha_key].shape
    img_shape     = scene.sensors()[0].film().crop_size()
    
    for idx_alpha, alpha in enumerate(alphas):
        print("  - {}/{}".format(idx_alpha + 1, len(alphas)), end='\r') 
        params[alpha_key] = mi.TensorXf(np.ones(texture_shape)*alpha)
        params.update()

        dr.enable_grad(params[alpha_key])

        grad_backward = mi.Float(0.0)
        if 'diff' in method:
            # Differential sampling strategy, use antithetic sampling.
            # Note that we use the same seed twice, and use half the number of samples for each pass.
            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp//2, antithetic_pass=False)
            dr.backward(image)

            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp//2, antithetic_pass=True)
            dr.backward(image)

            # Average accumulated gradients from both passes
            grad_backward = 0.5 * dr.grad(params[alpha_key])
        else:
            # Produce differentiable rendering
            image = mi.render(scene, params,
                              integrator=integrator, seed=0, spp=spp)

            # And propagate derivatives backwards through it
            dr.backward(image)
            grad_backward = dr.grad(params[alpha_key])

        # Save output gradient image
        outname = "{}/a_{:02d}.exr".format(method_dir, idx_alpha)
        mi.util.convert_to_bitmap(grad_backward, uint8_srgb=False).write(outname)
    
produce_plots(diff_methods, diff_method_names, 'gradients_reverse', gradients=True)